## Supporting Code Area ...  NO NEED TO READ THIS

In [9]:
#from flask import Flask
#app = Flask(__name__)

#@app.route('/')
#def index():
    #return render_template('index.html', variable=questionsoutput)
#Code for loading data in from a URL
import requests

#Get data from URL
def getURL(url,quiet=False):
    if not quiet: print(url)
    r=requests.get(url)
    if not quiet: print(r.status_code)
    return r
    

#Should build a proper recursive loader
def loader(url,quiet=True):
    items=[]
    done=False
    r=getURL(url,quiet)
    while not done:
        items=items+r.json()['result']['items']
        if 'next' in r.json()['result']:
            r=getURL(r.json()['result']['next']+'&_pageSize=500',quiet)
        else: done=True
    return items



In [10]:
#A tabluar data analysis package that can make life easier...
import pandas as pd


In [11]:
#Utilities for expressing natural time
import datetime
from dateutil.relativedelta import relativedelta

## Generate the API URL and Load the Data

#Quick peek at the API

#http://lda.data.parliament.uk/commonswrittenquestions.json?_view=Written+Questions&_pageSize=10&_page=0
stub='http://lda.data.parliament.uk'.strip('/')

if SINCE:
    url='{}/{}.json?{}'.format(stub,'commonswrittenquestions','min-dateTabled={}'.format(PERIOD))
else:
    url='{}/{}.json?dateTabled={}&{}'.format(stub,'commonswrittenquestions','2017-09-14','_pageSize=500')

#The API returns a list of written questions on/since the specified date
items=loader(url)
items[0]

In [16]:
stub='http://lda.data.parliament.uk'.strip('/')
url='{}/{}.json?dateTabled={}&{}'.format(stub,'commonswrittenquestions','2018-03-08','_pageSize=500')
  
#The API returns a list of written questions on/since the specified date
items=loader(url)
items[0]

{'AnswerDate': {'_datatype': 'dateTime', '_value': '2018-03-08'},
 'AnsweringBody': [{'_value': 'Church Commissioners'}],
 '_about': 'http://data.parliament.uk/resources/857659',
 'dateTabled': {'_datatype': 'dateTime', '_value': '2018-03-08'},
 'questionText': 'To ask the right hon. Member for Meriden, representing the Church Commissioners, what funding is available for church infrastructure projects.',
 'tablingMember': {'_about': 'http://data.parliament.uk/members/3985',
  'label': {'_value': 'Biography information for Robert Halfon'}},
 'tablingMemberPrinted': [{'_value': 'Robert Halfon'}],
 'title': 'House of Commons Tabled Parliamentary Question 2017/19 904239',
 'uin': '904239'}

## Count by Answering Body

We can now work through the answers and identify the answering body associated with each.

In [19]:
def getAnsweringBody(item):
    rel=[]
    for body in item['AnsweringBody']:
        rel.append(body['_value'])
    return rel
    

In [20]:
#Structurally in the API data, it may be the case that a question is directed at several Answering Bodies.
#We could treat these as multiple separate requests requiring tagging, once for each body:
#rels=[rel for s in [getAnsweringBody(q) for q in items] for rel in s]

#Or by default we just go with the first named body
rels=[rel for s in [getAnsweringBody(q) for q in items] for rel in s]

#Also flag where there were multiple bodies?
#multibody=[m for m in [getAnsweringBody(q)  for q in items] if len(m)>1]
#if multibody: print("Some requests appear to be targeted at multiple bodies:", m)

#Display the answering body for the first few questions
df=pd.DataFrame(rels)
df.columns=['Answering Body']
df

,Answering Body
0,Church Commissioners
1,"Department for Environment, Food and Rural Aff..."
2,"Department for Environment, Food and Rural Aff..."
3,"Department for Environment, Food and Rural Aff..."
4,"Department for Environment, Food and Rural Aff..."
5,Ministry of Defence
6,Department for Education
7,Foreign and Commonwealth Office
8,"Department for Environment, Food and Rural Aff..."
9,Department of Health and Social Care


We want to count the number of question referred to each answering body and allocate on that basis.

In [21]:
dfc=df.groupby('Answering Body').size().rename('Count').reset_index().sort_values('Count',ascending=False)
#dfc.head()

In [22]:
#This is a bit of utility code that helps us count how work is allocated
#https://github.com/timdiels/chicken_turtle_util/blob/master/chicken_turtle_util/algorithms.py
class _handler(object):
    def __init__(self,_name):
        self._name = _name
        self._items = []
        self._itemCounts = []
        self._count_sum = 0
        
    def add(self, item, count):
        self._items.append(item)
        self._itemCounts.append((item,count))
        self._count_sum += count
    
    @property
    def name(self):
        return self._name
    
    @property
    def items(self):
        return (self._items, self._count_sum)
    
    @property
    def itemCounts(self):
        return (self._itemCounts, self._count_sum)
    
    @property
    def count_sum(self):
        return self._count_sum

### YOUR TURN...

Who's doing the work? Are there any preferences?

In [23]:
#List of names of folk doing the work
handlers=['Clare','Ned','Jason','Bosede','Kirsty']

#Are there any requirements as to whom particular targeted answering bodies will specifically be allocated to?
handlerPrefs={}
#e.g. handlerPrefs={'Ann':['Department of Health','Attorney General'],
 #             'David':['Ministry of Defence']}


#Are there any requirements as to which particular targeted answering bodies must not be allocated to a particular person?
handlerAvoid={}

## From here on in, it's the machine...

Allocate heuristically - sort from largest job to lowest; give next job to person with least load overall.

If there are only a few questions in total, may want to allocate to a single person?

In [24]:
#This bit of code handles the allocation
handlerJobs=[_handler(h) for h in handlers]
handled=[]

#First of all, allocate according the preferences (actually, we treat these as *required* allocations)
for handler_ in handlerJobs: 
    if handler_.name in handlerPrefs:
        for ix,row in dfc[dfc['Answering Body'].isin(handlerPrefs[handler_.name])].iterrows():
            #Add a start to an answering body name of the allocation was required
            handler_.add(row['Answering Body']+'*',row['Count'])
            handled.append(row['Answering Body'])

#Allocating the work is an example of a multi-way partition problem.
#This sort of problem can be quite hard to solve exactly, but there are heuristics
#Eg allocate from largest job to lowest; give next job to person with least load overall
for ix,row in dfc[~dfc['Answering Body'].isin(handled)].iterrows():
    #Note the 'if not' tweak to the allocation so someone who wants to avoid an answering body actually does so...
    handler_ = min(handlerJobs, key=lambda handler_: handler_._count_sum if not (handler_.name in handlerAvoid and row['Answering Body'] in handlerAvoid[handler_.name])  else float('inf')  ) 
    handler_.add(row['Answering Body'],row['Count'])
    

In [28]:
questions=[]
#Or more prettily...
for h in handlerJobs:
    abtxt=''.join(['\n\t- {} for the {}'.format(b[1], b[0]) for b in h.itemCounts[0]])
    print('----\n{} needs to tag {} questions:{}'.format(h.name, h.itemCounts[1],abtxt))    

----
Clare needs to tag 36 questions:
	- 36 for the Department of Health and Social Care
----
Ned needs to tag 33 questions:
	- 24 for the Ministry of Justice
	- 7 for the Department for Transport
	- 1 for the Department for Exiting the European Union 
	- 1 for the Church Commissioners
----
Jason needs to tag 34 questions:
	- 17 for the Ministry of Defence
	- 8 for the Department for Education
	- 5 for the Ministry of Housing, Communities and Local Government
	- 4 for the Department for Digital, Culture, Media and Sport
----
Bosede needs to tag 33 questions:
	- 13 for the Department for Work and Pensions
	- 9 for the Foreign and Commonwealth Office
	- 8 for the Department for Business, Energy and Industrial Strategy
	- 2 for the Northern Ireland Office
	- 1 for the Prime Minister
----
Kirsty needs to tag 33 questions:
	- 11 for the Treasury
	- 9 for the Department for Environment, Food and Rural Affairs
	- 8 for the Home Office
	- 4 for the Cabinet Office
	- 1 for the Attorney General
